# Llama_Indexを試す

In [1]:
import llama_index
llama_index.__version__

'0.8.24.post1'

## PDFを読み込む
- 読み込み可能なアプリケーション
  - dvipdfmx
  - word
- 読み込みがうまくかないアプリケーション
  - Adobe
  - iText

- 使用データ
  - [STクラウドサーバーサービス契約約款](https://www.stnet.co.jp/cmmn/data/stcloud.pdf)
  - [インターネットセキュリティサービス契約約款
(https://www.stnet.co.jp/cmmn/data/inet.pdf)
  - [ビジネス光ネットサービス契約約款](https://www.stnet.co.jp/cmmn/data/oshigotonet.pdf)]

In [2]:
# from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader
from llama_index import download_loader

CJKPDFReader = download_loader("CJKPDFReader")
loader = CJKPDFReader()
documents = loader.load_data(file="pdf_data/data_0.pdf")
print(len(documents))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.6 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for cryptography>=36.0.0 from https://files.pythonhosted.org/packages/46/74/f9eba8c947f57991b5dd5e45797fdc68cc70e444c32e6b952b512d42aba5/cryptography-41.0.3-cp37-abi3-manylinux_2_28_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 15.6 MB/s eta 0:00:0000:0100:01


1


In [4]:
with open("./pdf_to_text/pdf_to_text_0.txt",encoding="utf-8",mode="w") as f:
    f.write(documents[0].text) 

In [5]:
documents[0].to_dict().keys()

dict_keys(['id_', 'embedding', 'metadata', 'excluded_embed_metadata_keys', 'excluded_llm_metadata_keys', 'relationships', 'hash', 'text', 'start_char_idx', 'end_char_idx', 'text_template', 'metadata_template', 'metadata_seperator', 'class_name'])

In [6]:
text_data = documents[0].text.split("\n\n")
text_data = [text.replace("\n","") for text in text_data] 

In [7]:
len(text_data)

1776

In [8]:
documents[0].text[:200]

'ＳＴクラウドサーバーサービス契約約款 \n\n２０２３年５月１５日 \n\n株式会社 ＳＴＮｅｔ \n\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n\x0c\n目    次 \n\n第１章  総則 \n\n第  １条  約款の適用・・・・・・・・・・・・・・・・・・・・・・・・・・・１ \n第  ２条  約款の変更・・・・'

## Qdrantの設定

In [3]:
import qdrant_client
import logging
from llama_index.storage.storage_context import StorageContext
import sys

client = qdrant_client.QdrantClient(host='qdrant', port=6333)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

vector_store = QdrantVectorStore(client=client, collection_name="")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

NameError: name 'QdrantVectorStore' is not defined

## ベクトル化
### note
- 文分割は読点等のsplit程度
- データごとに処理が必要

In [9]:
paragraph_separator = "\n\n" # 段落分割
chunk_size = 1024 #チャンク（トークン）数
chunk_overlap = 20 # 前のチャンクをどのくらい含めるか
secondary_chunking_regex = '[^,.．;。]+[,.．;。]?'# 文分割

In [10]:
from llama_index.text_splitter import SentenceSplitter
from llama_index.node_parser import SimpleNodeParser
from llama_index import VectorStoreIndex,ServiceContext
import tiktoken

# ノードパーサーの準備
text_splitter = SentenceSplitter(
    paragraph_separator=paragraph_separator,
    secondary_chunking_regex = secondary_chunking_regex,
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
)
node_parser = SimpleNodeParser.from_defaults(
    text_splitter=text_splitter
)
service_context = ServiceContext.from_defaults(
    node_parser=node_parser
)
service_context.node_parser

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


SimpleNodeParser(text_splitter=SentenceSplitter(chunk_size=1024, chunk_overlap=20, separator=' ', paragraph_separator='\n\n', secondary_chunking_regex='[^,.．;。]+[,.．;。]?', chunking_tokenizer_fn=<function sent_tokenize at 0x7ff0579eee80>, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7ff08c250f50>, tokenizer=functools.partial(<bound method Encoding.encode of <Encoding 'gpt2'>>, allowed_special='all')), include_metadata=True, include_prev_next_rel=True, metadata_extractor=None, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7ff056b6ecd0>)

In [11]:
node_list = service_context.node_parser.get_nodes_from_documents(documents)
enc = tiktoken.get_encoding("gpt2")
print(len(node_list))
for node in node_list:
    print(len(enc.encode(node.text)))
    print(len(node.text))
    print(node.text.replace("\n",""))
    break

108
1002
725
ＳＴクラウドサーバーサービス契約約款 ２０２３年５月１５日 株式会社 ＳＴＮｅｔ                                      目    次 第１章  総則 第  １条  約款の適用・・・・・・・・・・・・・・・・・・・・・・・・・・・１ 第  ２条  約款の変更・・・・・・・・・・・・・・・・・・・・・・・・・・・１ 第  ３条  用語の定義・・・・・・・・・・・・・・・・・・・・・・・・・・・１ 第２章  ＳＴクラウドサーバーサービスの提供区域 第  ４条  ＳＴクラウドサーバーサービスの提供区域・・・・・・・・・・・・・２ 第３章  契約 第  ５条  契約の単位・・・・・・・・・・・・・・・・・・・・・・・・・・・２ 第  ６条  ＳＴクラウドサーバー契約申込の方法・・・・・・・・・・・・・・・２ 第  ７条  ＳＴクラウドサーバー契約申込の承諾・・・・・・・・・・・・・・・２ 第  ８条  最低利用期間・・・・・・・・・・・・・・・・・・・・・・・・・・２ 第  ９条  ＳＴクラウドサーバー契約内容の変更・・・・・・・・・・・・・・・２ 第１０条  サービス利用権の譲渡禁止・・・・・・・・・・・・・・・・・・・・３ 第１１条  ＳＴクラウドサーバー契約者が行うＳＴクラウドサーバー契約の解除・３ 第１２条  当社が行うＳＴクラウドサーバー契約の解除・・・・・・・・・・・・３     第１３条  その他の提供条件・・・・・・・・・・・・・・・・・・・・・・・・３


In [ ]:
index = VectorStoreIndex.from_documents(documents=documents,vector_store=vector_store,service_context=service_context)